In [88]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
load_dotenv()
API_KEY = os.getenv('API_KEY')
import time
import schedule

In [89]:
def get_all_sports():
    """
    This function gets all the sports from odds api and returns it

    Returns:
        sports: a list of sports on avalible with the odds api
    """
    
    sports_url = f'https://api.the-odds-api.com/v4/sports/?apiKey={API_KEY}'

    sports_response = requests.get(sports_url)
    sports_data = sports_response.json()

    sports = []
    for sport in sports_data:
        sports.append(sport['key'])

    return sports

def get_sport_teams_odds(sport):
    """gets the different teams playing and sports book odds for each team given the parameter sport

    Args:
        sport (String): the sport we want to get the team odds for

    Returns:
        sport_team_odds: returns a list of tubles, containing the teams playing, followed by a pd df
        containing the sports books, and odds for each team for each sports book. We have n tubles for n 
        teams playing in the sport
        
    """
    

    sport_team_odds_url = f'https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey={API_KEY}&regions=us&markets=h2h,spreads&oddsFormat=american'
    response = requests.get(sport_team_odds_url)
    data = response.json()
    
    if len(data) <= 2:
        return None
    
    sport_team_odds = []
    for team in data:
        
        sports_teams = f"{team['bookmakers'][0]['markets'][0]['outcomes'][0]['name']} vs {team['bookmakers'][0]['markets'][0]['outcomes'][1]['name']}"
        
        books = []
        odds1 = []
        odds2 = []
        for odds in team['bookmakers']:
            books.append(odds['title'])
            odds1.append(odds['markets'][0]['outcomes'][0]['price'])
            odds2.append(odds['markets'][0]['outcomes'][1]['price']) 
        
        data = {
                'Books': books,
                'Odds1': odds1,
                'Odds2': odds2
            }
        team_odds = (sports_teams, pd.DataFrame(data))
        sport_team_odds.append(team_odds)
    return sport_team_odds

def calc_arb_opp(sport_odds):
    
    for game_index in range(len(sport_odds)):
        # print(sport_odds[game_index][0])
        max_odds1 = sport_odds[game_index][1]['Odds1'].max()
        max_odds2 = sport_odds[game_index][1]['Odds2'].max()
        
        implied_prob = find_probs(max_odds1, max_odds2) 
        max_odds1_book = sport_odds[game_index][1].loc[sport_odds[game_index][1]['Odds1'].idxmax(), 'Books']
        max_odds2_book = sport_odds[game_index][1].loc[sport_odds[game_index][1]['Odds2'].idxmax(), 'Books']
        if implied_prob < 1:
            print('\n')
            print(f'Arbitrage Opportunity Avalible:{implied_prob} | Game: {sport_odds[game_index][0]}  {max_odds1_book}:{max_odds1} | {max_odds2_book}:{max_odds2}')
            print('\n')
        else:
            print(f'NO SBA: {implied_prob} | Game: {sport_odds[game_index][0]}   {max_odds1_book}:{max_odds1} | {max_odds2_book}:{max_odds2}')
def find_probs(b1, b2):
    
    if b1 < 0:
        p1 = b1 / (b1 - 100)
    else:
        p1 = 100 / (b1 + 100)
        
    if b2 < 0:
        p2 = b2 / (b2 - 100)
    else:
        p2 = 100 / (b2 + 100)
        
    return p1 + p2

In [90]:
sports = [ 'baseball_kbo', 'baseball_mlb', 'baseball_mlb_world_series_winner', 'baseball_ncaa', 'baseball_npb', 'basketball_euroleague', 'basketball_nba', 'basketball_nba_championship_winner', 'basketball_wnba', 'golf_masters_tournament_winner', 'golf_pga_championship_winner', 'golf_the_open_championship_winner', 'golf_us_open_winner', 'icehockey_nhl', 'icehockey_nhl_championship_winner', 'icehockey_sweden_allsvenskan', 'icehockey_sweden_hockey_league', 'politics_us_presidential_election_winner', 'rugbyleague_nrl']

In [150]:
nba_odds = get_sport_teams_odds('basketball_nba')

In [159]:
knicks_IO = nba_odds[1][1]
print(knicks_IO)
nba_odds[1][0]


             Books  Odds1  Odds2  Minute
0       DraftKings   -185    154       0
1          FanDuel   -188    158       0
2   PointsBet (US)   -190    160       0
3        LowVig.ag   -170    150       0
4     BetOnline.ag   -170    150       0
5           BetMGM   -185    150       0
6          WynnBET   -185    158       0
7          Caesars   -190    158       0
8        BetRivers   -182    155       0
9           Unibet   -182    155       0
10       SuperBook   -190    165       0
11     MyBookie.ag   -179    151       0
12          Bovada   -180    155       0
13           BetUS   -180    155       0


'Dallas Mavericks vs Los Angeles Clippers'

In [152]:
knicks_IO['Minute'] = [0] * 14

# Melt the DataFrame to create separate rows for each 'Books' and 'Odds' pair
melted_df = pd.melt(knicks_IO, id_vars=['Books', 'Minute'], value_vars=['Odds1', 'Odds2'], var_name='Team', value_name='Odds')

# Create a MultiIndex from the 'Books' and 'Team' columns
melted_df.set_index(['Books', 'Team', 'Minute'], inplace=True)

melted_df = melted_df.sort_index(level=['Books', 'Team'])
# Reshape the DataFrame to have 'Books' and 'Team' as columns and 'Minute' as index
nba_game_odds = melted_df.unstack(level=['Books', 'Team'])

# Drop the first level of the MultiIndex ('Odds') from the columns
nba_game_odds.columns = nba_game_odds.columns.droplevel(0)

# Rename the columns to match the desired format
nba_game_odds.columns = pd.MultiIndex.from_tuples([(book, team) for book in nba_game_odds.columns.levels[0] for team in nba_game_odds.columns.levels[1]], names=['Sportsbook', 'Team'])
nba_game_odds

Sportsbook BetMGM       BetOnline.ag       BetRivers       BetUS       Bovada  \
Team        Odds1 Odds2        Odds1 Odds2     Odds1 Odds2 Odds1 Odds2  Odds1   
Minute                                                                          
0            -185   150         -170   150      -182   155  -180   155   -180   

Sportsbook        ... MyBookie.ag       PointsBet (US)       SuperBook        \
Team       Odds2  ...       Odds1 Odds2          Odds1 Odds2     Odds1 Odds2   
Minute            ...                                                          
0            155  ...        -179   151           -190   160      -190   165   

Sportsbook Unibet       WynnBET        
Team        Odds1 Odds2   Odds1 Odds2  
Minute                                 
0            -182   155    -185   158  

[1 rows x 28 columns]

In [166]:

new_entry = pd.DataFrame({
    'Books': [
        'DraftKings', 'FanDuel', 'PointsBet (US)', 'LowVig.ag',
        'BetOnline.ag', 'BetMGM', 'WynnBET', 'Caesars',
        'BetRivers', 'Unibet', 'SuperBook', 'MyBookie.ag',
        'Bovada', 'BetUS'
    ],
    'Odds1': [
        -185, -188, -190, -170, -170, -185, -185, -190, -182,
        -182, -190, -179, -180, -180
    ],
    'Odds2': [
        154, 158, 160, 150, 150, 150, 158, 158, 155, 155, 165,
        151, 155, 155
    ]
})
new_entry['Minute'] = [5] * len(new_entry)
new_entry = pd.melt(new_entry, id_vars=['Books', 'Minute'], value_vars=['Odds1', 'Odds2'], var_name='Team', value_name='Odds')
new_entry.set_index(['Books', 'Team', 'Minute'], inplace=True)
new_entry = new_entry.sort_index(level=['Books', 'Team'])
new_entry = new_entry.unstack(level=['Books', 'Team'])
new_entry

# combined = pd.concat([nba_game_odds, new_entry], axis=0)
# combined

Odds                                                              \
Books  BetMGM       BetOnline.ag       BetRivers       BetUS       Bovada   
Team    Odds1 Odds2        Odds1 Odds2     Odds1 Odds2 Odds1 Odds2  Odds1   
Minute                                                                      
5        -185   150         -170   150      -182   155  -180   155   -180   

              ...                                                         \
Books         ... MyBookie.ag       PointsBet (US)       SuperBook         
Team   Odds2  ...       Odds1 Odds2          Odds1 Odds2     Odds1 Odds2   
Minute        ...                                                          
5        155  ...        -179   151           -190   160      -190   165   

                                   
Books  Unibet       WynnBET        
Team    Odds1 Odds2   Odds1 Odds2  
Minute                             
5        -182   155    -185   158  

[1 rows x 28 columns]

In [5]:
def job():
    print("Running the job...")

# Schedule the job to run every 5 minutes
schedule.every(1).minutes.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

Running the job...
Running the job...
Running the job...
Running the job...


KeyboardInterrupt: 